# Task 3: Identify the three most frequent 311 complaint types by borough. 
- Are the same complaint types frequent in all five boroughs of the City? 
- How might you explain the differences? 
- How does the distribution of complaints change over time for certain neighborhoods and how could this be explained?

### Data Souce: https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9
### The data 311.csv used here is the newest version.

# Load Libraries

In [1]:
from datetime import datetime, timedelta
import time
import modin.pandas as pd
# use modin to accelerate I/O process
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
#import mpld3
#from pyspark.sql import SparkSession
# for cluster
%matplotlib inline

Dask needs bokeh >= 0.13.0 for the dashboard.
Continuing without the dashboard.


# Load the data (12.5GB) and trim it

In [2]:
# Load it into a Dataframe using pandas
path = 'data/311.csv'
df = pd.read_csv(path)
df.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,20934224,07/29/2011 07:51:03 PM,08/29/2011 08:43:43 PM,DPR,Department of Parks and Recreation,Damaged Tree,Branch or Limb Has Fallen Down,Street,11432,82-21 DONCASTER PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.720235,-73.789196,"(40.72023539094363, -73.78919639527908)"
1,20934225,07/29/2011 08:34:33 PM,07/30/2011 05:38:24 AM,DOT,Department of Transportation,Broken Muni Meter,No Receipt,Street,10022,399 PARK AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.759540,-73.972245,"(40.75954048841318, -73.97224520229886)"
2,20934226,07/29/2011 09:02:46 PM,08/02/2011 12:29:17 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Rodent Sighting,Park,10009,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20934227,07/29/2011 10:20:18 PM,07/29/2011 11:17:35 PM,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,11234,5804 FARRAGUT ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.636557,-73.920942,"(40.636556605492814, -73.92094245293143)"
4,20934228,07/29/2011 10:35:13 PM,07/30/2011 12:30:20 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10460,1723 TAYLOR AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.842509,-73.869199,"(40.84250861667027, -73.8691992447988)"


In [3]:
df.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [4]:
len(df.columns)

41

In [5]:
# Drop columns not needed
df.drop(df.columns[[0,4,8,9,10,11,12,13,14,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,40]], axis=1, inplace=True)

In [6]:
df

,Created Date,Closed Date,Agency,Complaint Type,Descriptor,Location Type,City,Latitude,Longitude
0,07/29/2011 07:51:03 PM,08/29/2011 08:43:43 PM,DPR,Damaged Tree,Branch or Limb Has Fallen Down,Street,JAMAICA,40.720235,-73.789196
1,07/29/2011 08:34:33 PM,07/30/2011 05:38:24 AM,DOT,Broken Muni Meter,No Receipt,Street,NEW YORK,40.759540,-73.972245
2,07/29/2011 09:02:46 PM,08/02/2011 12:29:17 PM,DPR,Maintenance or Facility,Rodent Sighting,Park,NEW YORK,NaN,NaN
3,07/29/2011 10:20:18 PM,07/29/2011 11:17:35 PM,NYPD,Noise - Residential,Loud Talking,Residential Building/House,BROOKLYN,40.636557,-73.920942
4,07/29/2011 10:35:13 PM,07/30/2011 12:30:20 AM,NYPD,Noise - Residential,Loud Music/Party,Residential Building/House,BRONX,40.842509,-73.869199
...,...,...,...,...,...,...,...,...,...
21923948,07/29/2011 03:38:31 PM,08/05/2011 04:26:11 PM,DHS,DHS Advantage - Tenant,Other Issue,Tenant Address,BRONX,NaN,NaN
21923949,07/29/2011 03:52:13 PM,08/12/2011 03:04:52 PM,DOF,SCRIE,SCRIE Assistance,Senior Address,BRONX,NaN,NaN
21923950,07/29/2011 03:59:39 PM,07/30/2011 08:23:44 AM,DOT,Highway Condition,Pothole - Highway,Highway,BRONX,40.836810,-73.873710
21923951,07/29/2011 05:24:57 PM,08/03/2011 10:50:14 AM,DOT,Broken Muni Meter,No Receipt,Street,FOREST HILLS,40.720102,-73.843469


In [7]:
# Check df size
df.shape

(21923953, 9)

In [13]:
# Create a new csv 1GB
df.to_csv('data/311_not_filtered.csv', index=False )

To request implementation, send an email to feature_requests@modin.org.


In [14]:
del df
df1 = pd.read_csv('data/311_not_filtered.csv')
df1.head()
df2 = df1.dropna()

In [15]:
df2

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:55047'], apply_list_of_funcs-d54c3cbaa1d6280b4e25fef6902a2cb9
NoneType: None
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {'apply_list_of_funcs-d54c3cbaa1d6280b4e25fef6902a2cb9': ('tcp://127.0.0.1:55047',)}


,Created Date,Closed Date,Agency,Complaint Type,Descriptor,Location Type,City,Latitude,Longitude
0,07/29/2011 07:51:03 PM,08/29/2011 08:43:43 PM,DPR,Damaged Tree,Branch or Limb Has Fallen Down,Street,JAMAICA,40.720235,-73.789196
1,07/29/2011 08:34:33 PM,07/30/2011 05:38:24 AM,DOT,Broken Muni Meter,No Receipt,Street,NEW YORK,40.759540,-73.972245
3,07/29/2011 10:20:18 PM,07/29/2011 11:17:35 PM,NYPD,Noise - Residential,Loud Talking,Residential Building/House,BROOKLYN,40.636557,-73.920942
4,07/29/2011 10:35:13 PM,07/30/2011 12:30:20 AM,NYPD,Noise - Residential,Loud Music/Party,Residential Building/House,BRONX,40.842509,-73.869199
5,07/29/2011 11:46:33 PM,07/30/2011 12:07:47 AM,NYPD,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,BROOKLYN,40.704048,-73.934250
...,...,...,...,...,...,...,...,...,...
21923943,07/30/2011 02:11:03 AM,07/30/2011 05:19:53 AM,NYPD,Noise - Residential,Loud Music/Party,Residential Building/House,BRONX,40.880376,-73.885937
21923944,07/30/2011 02:18:32 AM,07/30/2011 04:09:12 AM,NYPD,Noise - Residential,Loud Music/Party,Residential Building/House,BROOKLYN,40.672260,-73.941917
21923950,07/29/2011 03:59:39 PM,07/30/2011 08:23:44 AM,DOT,Highway Condition,Pothole - Highway,Highway,BRONX,40.836810,-73.873710
21923951,07/29/2011 05:24:57 PM,08/03/2011 10:50:14 AM,DOT,Broken Muni Meter,No Receipt,Street,FOREST HILLS,40.720102,-73.843469


In [16]:
df2.to_csv('data/311_filtered1.csv', index=False )